In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mph
import pprint

from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / 'util/'))

from functions import *
from materials import *

config = load_config( "config/CoaxCavity.yaml" )

In [3]:
client = mph.start()
pymodel = client.create('CoaxCavity')

In [4]:
parameters = pymodel/'parameters'
(parameters/'Parameters 1').rename('Parameters')

for parameter in config['parameter']:
    value = config['parameter'][parameter]
    pymodel.parameter( parameter, value)

In [5]:
model = pymodel.java
comp1 = model.component().create("comp1", True)

geom1 = comp1.geom().create("geom1", 3) # 3 means 3D
geom1.lengthUnit("mm")
geom1.run()


# Make cavity box 
block1 = geom1.create("block1", "Block")
block1.set("pos", ["-box_length/2", "-box_length/2", "outer_height - box_height"])
block1.set("size", ["box_length", "box_length", "box_height"])
geom1.run("block1")

# Work plane 1
wp1 = geom1.create("wp1", "WorkPlane")
wp1.set("unite", True)
c1 = wp1.geom().create("c1", "Circle")
c1.set("pos",  [0., 0.] )
c1.set("r", 'outer_radius')
wp1.geom().run("c1")

# Extrude 1
ext1 = geom1.create("ext1", "Extrude")
ext1.set("workplane", "wp1")
ext1.selection("input").set("wp1")
ext1.setIndex("distance", 'outer_height', 0) # 0 means first row ?
geom1.run("ext1")


# Work plane 2
wp2 = geom1.create("wp2", "WorkPlane")
wp2.set("unite", True)
c2 = wp2.geom().create("c2", "Circle")
c2.set("pos",  [0., 0.] )
c2.set("r", 'coax_radius')
wp2.geom().run("c2")

# Extrude 2
ext2 = geom1.create("ext2", "Extrude")
ext2.set("workplane", "wp2")
ext2.selection("input").set("wp2")
ext2.setIndex("distance", 'coax_height', 0)
geom1.run("ext2")

# Subtract
diff1 = geom1.create("diff1", "Difference")
diff1.selection("input").set("block1")
diff1.selection("input2").set("ext1")
geom1.run("diff1")

# Union 1
uni1 = geom1.create("uni1", "Union")
uni1.selection("input").set("diff1", "ext2")
uni1.set("intbnd", False)
geom1.run("uni1")



# Block
boundary = geom1.create("boundary", "Block")
boundary.set("pos", ["-box_length/2", "-box_length/2", "outer_height - box_height"])
boundary.set("size", ["box_length", "box_length", "box_height"])
geom1.run("boundary")

# Subtract
air = geom1.create("air", "Difference")
air.selection("input").set("boundary")
air.selection("input2").set("uni1")
air.set("keepsubtract", True)
geom1.run("air")


# Work plane 6
wp6 = geom1.create("wp6", "WorkPlane")
wp6.set("unite", True)
wp6.set("quickz", "chip_pos_z - wafer_height")
r3 = wp6.geom().create("r3", "Rectangle")
r3.set("base", "center")
r3.set("pos", ['0', 'chip_pos_y'])
r3.set("size", ['wafer_width', 'wafer_length'])
wp6.geom().run("r3")
wp6.label("Chip")
geom1.run("wp6")

# Extrude 6
ext6 = geom1.create("ext6", "Extrude")
ext6.set("workplane", "wp6")
ext6.selection("input").set("wp6")
ext6.setIndex("distance", 'wafer_height', 0)
geom1.run("ext6")

# Work plane 7
wp7 = geom1.create("wp7", "WorkPlane")
wp7.set("unite", True)
wp7.set("quickz", "chip_pos_z")
r4 = wp7.geom().create("r4", "Rectangle")
r4.set("size", ['chip_width', 'chip_height'])
r4.set("base", "center")
r4.set("pos", ['0', 'chip_pos_y + chip_height/2 + chip_gap/2'])
wp7.geom().run("r4")

fil3 = wp7.geom().create("fil3", "Fillet")
fil3.selection("pointinsketch").set("r4", 1, 2, 3, 4)
fil3.set("radius", 'chip_fillet')
wp7.geom().run("fil3")

r5 = wp7.geom().create("r5", "Rectangle")
r5.set("base", "center")
r5.set("pos", ['0', 'chip_pos_y + jj_height/2 + finger_height/2'])
r5.set("size", ["jj_width", "finger_height"])
wp7.geom().run("r5")

# Union 2
uni2 = wp7.geom().create("uni2", "Union")
uni2.selection("input").set("fil3", "r5")
uni2.set("intbnd", False)
wp7.geom().run("uni2")

mir2 = wp7.geom().create("mir2", "Mirror")
mir2.selection("input").set("uni2")
mir2.set("pos", ['0', 'chip_pos_y'])
mir2.set("keep", True)
mir2.set("axis", [0., 1.])
wp7.geom().run("mir2")
wp7.label("Transmon pad")

r6 = wp7.geom().create("r6", "Rectangle")
r6.set("base", "center")
r6.set("size", ['jj_width','jj_height'])
r6.set("pos", ['0', 'chip_pos_y'])
wp7.geom().run("r6")

geom1.run("wp7")


<java object 'com.comsol.clientapi.impl.GeomSequenceClient'>

In [6]:
# run geometry and set view after making everything
geom1.run()
comp1.view("view1").set("transparency", True)

<java object 'com.comsol.clientapi.impl.ViewClient'>

In [7]:
air.set("selresult", True)
uni1.set("selresult", True)
ext6.set("selresult", True)
wp7.set("selresult", True)
air.set("selresultshow", "all")
uni1.set("selresultshow", "all")
ext6.set("selresultshow", "all")
wp7.set("selresultshow", "all")

<java object 'com.comsol.clientapi.impl.WorkPlaneFeatureClient'>

## Assign material

In [8]:
assign_material(node = comp1, material = "air", tag = "mat3", name = "Air", selection = "geom1_air_dom") # selection = 2
assign_material(node = comp1, material = "copper", tag = "mat2", name = "Copper", selection = "geom1_uni1_dom") # selection = 1
assign_material(node = comp1, material = "silicon", tag = "mat1", name = "Silicon", selection = "geom1_ext6_dom") # selection = 3

In [9]:
mesh1 = comp1.mesh().create("mesh1")
emw  = comp1.physics().create("emw", "ElectromagneticWaves", "geom1")

In [10]:
# pec1 is always defined as default
pec2 = emw.create("pec2", "PerfectElectricConductor", 2)
pec2.selection().named("geom1_uni1_bnd") # bnd means boundary?

pec3 = emw.create("pec3", "PerfectElectricConductor", 2)
pec3.selection().named("geom1_wp7_bnd")

lelement1 = emw.create("lelement1", "LumpedElement", 2)
lelement1.selection().set(20)
lelement1.set("LumpedElementType", "Inductor")
lelement1.set("Lelement", "chip_inductance")

<java object 'com.comsol.clientapi.physics.impl.PhysicsFeatureClient'>

In [11]:
mesh1.run()

In [ ]:
std1 = model.study().create("std1")
eig = std1.create("eig", "Eigenfrequency")
eig.set("linpsolnum", "auto")
eig.set("solnum", "auto")
eig.set("notsolnum", "auto")
# eig.set("outputmap", [""])
eig.set("ngenAUX", "1")
eig.set("goalngenAUX", "1")
eig.set("ngenAUX", "1")
eig.set("goalngenAUX", "1")
eig.setSolveFor("/physics/emw", True)
eig.set("shift", "5[GHz]")
# eig.set("neigs", 4) # Number of eigen modes
# eig.set("eigwhich", "si") # Search for smallest imaginary part


<java object 'com.comsol.clientapi.impl.StudyFeatureClient'>

In [13]:
std1.createAutoSequences("all")
model.sol("sol1").runAll()

In [14]:
pg1 = model.result().create("pg1", "PlotGroup3D")
pg1.label("Electric Field (emw)")
pg1.set("data", "dset1")
pg1.set("frametype", "spatial")
pg1.set("showlegendsmaxmin", True)
pg1.selection().named("geom1_air_dom")

mslc1 = pg1.feature().create("mslc1", "Multislice")
mslc1.label("Multislice")
mslc1.set("showsolutionparams", "on")
mslc1.set("smooth", "internal")
mslc1.set("showsolutionparams", "on")
mslc1.set("data", "parent")
mslc1.set("znumber", "0")
mslc1.set("ynumber", "0")
mslc1.set("colortable", "RainbowClassic")
mslc1.set("colorscalemode", "logarithmic")
mslc1.feature().create("filt1", "Filter")
mslc1.feature("filt1").set("expr", "!isScalingSystemDomain")


gev1 = model.result().numerical().create("gev1", "EvalGlobal")
gev1.label("Eigenfrequencies (emw)")
gev1.set("data", "dset1")
gev1.set("expr", ["emw.freq", "emw.Qfactor"])
gev1.set("unit", ["GHz", "1"])

tbl1 = model.result().table().create("tbl1", "Table")
gev1.set("table", "tbl1")
gev1.run()
gev1.setResult()

pg1.run()


In [15]:
pymodel.save('mph/CoaxCavity.mph')
client.remove(pymodel)